<a href="https://colab.research.google.com/github/monoramasn/speech-recognition/blob/main/MMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git@main
!pip install -U accelerate
!pip install evaluate
!pip install jiwer
!pip install --upgrade transformers bitsandbytes datasets torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Succes

In [ ]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
import argparse
import evaluate
from scipy import signal
import numpy as np
from dataclasses import dataclass
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from typing import Any, Dict, List, Union
from sklearn.model_selection import train_test_split
from datasets import load_dataset, DatasetDict, Audio, load_from_disk, concatenate_datasets
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperTokenizer, WhisperFeatureExtractor
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
# Load datasets
edacc_dev = load_dataset("edinburghcstr/edacc", split="validation")
edacc_test = load_dataset("edinburghcstr/edacc", split="test")

# Create directories for each language in the test, train, and validation sets
base_dir = "Edacc_data"
os.makedirs(base_dir, exist_ok=True)

languages = edacc_test.unique("l1")
for split in ['test', 'train', 'validation']:
    for lang in languages:
        os.makedirs(os.path.join(base_dir, split, lang), exist_ok=True)

# Function to save audio data and corresponding text
def save_audio(example, folder):
    lang_folder = os.path.join(folder, example["l1"])
    os.makedirs(lang_folder, exist_ok=True)
    audio_filename = example['audio']['path'].split('/')[-1]
    audio_path = os.path.join(lang_folder, audio_filename)
    # Save audio file
    torchaudio.save(audio_path, torch.tensor(example['audio']['array']).unsqueeze(0), example['audio']['sampling_rate'])
    # Save corresponding text file
    text_path = os.path.join(lang_folder, f"{audio_filename}.txt")
    with open(text_path, "w") as f:
        f.write(example["text"])
    print(f"Saved audio: {audio_path}")
    print(f"Saved text: {text_path}")

# Save test audio and text to respective language directories
print("Saving test data...")
for example in edacc_test:
    save_audio(example, os.path.join(base_dir, 'test'))

# Split the validation dataset into train and validation sets
X = range(len(edacc_dev))
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

train_dataset = edacc_dev.select(X_train)
val_dataset = edacc_dev.select(X_val)

# Save train audio and text to respective language directories
print("Saving train data...")
for example in train_dataset:
    save_audio(example, os.path.join(base_dir, 'train'))

# Save validation audio and text to respective language directories
print("Saving validation data...")
for example in val_dataset:
    save_audio(example, os.path.join(base_dir, 'validation'))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# Initialize processor and model
model_name_or_path = "facebook/mms-1b-all"  # Replace with the actual MMS model identifier
processor = AutoProcessor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path)

# Function to downsample audio samples to 16 kHz
def downsample_audio(audio_array, source_sr, target_sr=16000):
    waveform = torch.tensor(audio_array, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
    resampled_waveform = torchaudio.transforms.Resample(source_sr, target_sr)(waveform)
    resampled_audio = resampled_waveform.squeeze(0).numpy()  # Remove channel dimension
    return resampled_audio

# Prepare the dataset
def prepare_dataset(batch):
    audio = batch["audio"]
    if audio["sampling_rate"] != 16000:
        audio["array"] = downsample_audio(audio["array"], audio["sampling_rate"], 16000)
        audio["sampling_rate"] = 16000

    input_values = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    with processor.as_target_processor():
        labels = processor(batch["text"]).input_ids

    batch["input_values"] = input_values
    batch["labels"] = labels
    batch["l1"] = batch["l1"]
    batch["audio"] = audio
    batch["text"] = batch["text"]

    return batch


In [ ]:
# Apply preprocessing
train_dataset = train_dataset.map(prepare_dataset, remove_columns=["audio"])
val_dataset = val_dataset.map(prepare_dataset, remove_columns=["audio"])

In [ ]:
from transformers import Trainer, TrainingArguments

# Data collator
@dataclass
class DataCollatorCTCWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_values = [feature["input_values"] for feature in features]
        labels = [feature["labels"] for feature in features]

        batch = self.processor.pad({"input_values": input_values, "labels": labels}, return_tensors="pt")

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor)


In [ ]:
import evaluate
metric = evaluate.load("wer")

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    warmup_steps=100,
    num_train_epochs=3,
    evaluation_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=4,
    logging_steps=600,
    save_steps=600,
    save_total_limit=2,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=processor
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
600,0.441100,0.336967
1200,0.305100,0.312929
1800,0.289000,0.287239
2400,0.271300,0.275187
3000,0.254000,0.263606
3600,0.254900,0.253147


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfi

TrainOutput(global_step=3939, training_loss=0.2983832861693809, metrics={'train_runtime': 4196.6588, 'train_samples_per_second': 1.877, 'train_steps_per_second': 0.939, 'total_flos': 2.27347578372096e+18, 'train_loss': 0.2983832861693809, 'epoch': 1.0})

In [ ]:
# Save the model and processor
model.save_pretrained("./saved_mms_model")
processor.save_pretrained("./saved_mms_model")

In [ ]:
import os
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
from datasets import load_metric
import torchaudio

# Load the saved model and processor
model = Wav2Vec2ForCTC.from_pretrained("./saved_mms_model")
processor = Wav2Vec2Processor.from_pretrained("./saved_mms_model")

# Function to downsample audio
def downsample_audio(audio_array, source_sr, target_sr=16000):
    waveform = torch.tensor(audio_array, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
    resampled_waveform = torchaudio.transforms.Resample(source_sr, target_sr)(waveform)
    resampled_audio = resampled_waveform.squeeze(0).numpy()  # Remove channel dimension
    return resampled_audio

# Function to evaluate WER for each language folder
def evaluate_language_wer(folder_path, processor, model, batch_size=16):
    wer_metric = load_metric("wer", trust_remote_code=True)
    audio_files = [f for f in os.listdir(folder_path) if f.endswith(".wav")]
    predictions = []
    references = []

    for i in range(0, len(audio_files), batch_size):
        batch_files = audio_files[i:i+batch_size]
        input_values = []
        for file in batch_files:
            audio_path = os.path.join(folder_path, file)
            text_path = audio_path + ".txt"
            with open(text_path, "r") as f:
                references.append(f.read().strip())
            audio, sampling_rate = torchaudio.load(audio_path)
            audio = audio.squeeze().numpy()
            if sampling_rate != 16000:
                audio = downsample_audio(audio, sampling_rate)
            features = processor(audio, sampling_rate=16000).input_values[0]
            input_values.append(features)

        input_values = torch.tensor(input_values).to(model.device)
        with torch.no_grad():
            logits = model(input_values).logits
        pred_ids = torch.argmax(logits, dim=-1)
        transcriptions = processor.batch_decode(pred_ids)
        predictions.extend(transcriptions)

    wer = wer_metric.compute(predictions=predictions, references=references)
    return wer

# Directory where test data is organized
base_dir = "organized_data"
languages = edacc_test.unique("l1")

# Evaluate and print WER for each language
for lang in languages:
    folder_path = os.path.join(base_dir, 'test', lang)
    wer = evaluate_language_wer(folder_path, processor, model)
    print(f"Language: {lang}, WER: {wer:.4f}")
